## Imports

In [1]:
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import requests
import json
warnings.filterwarnings("ignore")

In [2]:
import pyprind
from PyLyrics import *

## Read data from csv

In [3]:
df = pd.read_csv('songdata.csv')

In [4]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


## Add tags column

In [5]:
df['tags']=pd.Series('', index=df.index)

In [6]:
df.head()

,artist,song,link,text,tags
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...",
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...",
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,


## Randomly shuffling the data since artists are in alphabetical order

In [7]:
df = df.sample(frac=1).reset_index(drop=True)

In [8]:
df.head()

,artist,song,link,text,tags
0,Patti Smith,Come Back Little Sheba,/p/patti+smith/come+back+little+sheba_20105291...,By patti smith \n \nCome back little sheba ...,
1,David Guetta,Beautiful People Say,/d/david+guetta/beautiful+people+say_21063073....,"Hands out stretch, what you got \nGive me you...",
2,David Bowie,Day In Day Out,/d/david+bowie/day+in+day+out_20036847.html,Day-In Day-Out \nStay-In Fade-Out \n \nDay-...,
3,Wanda Jackson,The Right To Love,/w/wanda+jackson/the+right+to+love_20304993.html,They say we're young and undecided that we sho...,
4,Insane Clown Posse,Just Like That,/i/insane+clown+posse/just+like+that_20067652....,Just Like That \n[Violent J] \nJump out of b...,


## Scraping last.fm to get top tags associated with each song in our dataset

In [11]:
def getSongTags(artist,track):
    url = "http://ws.audioscrobbler.com/2.0/?method=track.getTopTags&api_key=159a3eba077ee16b23d096402ad28d47&artist="+artist+"&track="+track+"&format=json"
#     print(url)
    results = requests.get(url).json()
#     print(results)
    tagList = []
    if 'toptags' in results:
        toptags = results['toptags']
        if 'tag' in toptags:
            taglistss = toptags['tag']           
            for tagItem in taglistss:
                tagList.append(tagItem['name']) 
    return tagList

In [12]:
pbar = pyprind.ProgBar(df.shape[0])
for row_id in df.index:
    tags = getSongTags(df.loc[row_id]['artist'],df.loc[row_id]['song'])    
    df.loc[row_id,'tags'] = tags
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 06:15:18


In [13]:
df

,artist,song,link,text,tags
0,Britney Spears,Hang Up,/b/britney+spears/hang+up_20642448.html,"Baby, I'm Lying all alone, \nThe pillow is al...",[]
1,Kiss,Shock Me,/k/kiss/shock+me_20079711.html,Your lightnin's all I need \nMy satisfaction ...,"[hard rock, classic rock, rock, glam rock, 70s..."
2,Ugly Kid Joe,Would You Like To Be There,/u/ugly+kid+joe/would+you+like+to+be+there_101...,"Somewhere the evening sun \nIs falling down, ...","[90s, rock, hard rock, Best song on the album]"
3,Allman Brothers Band,Rockin' Horse,/a/allman+brothers+band/rockin+horse_20765345....,"""My gaurdian angel wears a hard hat,"" \nSaid ...","[Southern Rock, classic rock, blues rock, rock..."
4,Who,The Seeker,/w/who/the+seeker_20146792.html,I've looked under chairs \nI've looked under ...,"[classic rock, rock, 70s, british, hard rock, ..."
5,Vanilla Ice,Hit'em Hard,/v/vanilla+ice/hitem+hard_20143692.html,[Verse 1] \nHere comes the lyrical breakdown ...,[]
6,Kris Kristofferson,Not Everyone Knows,/k/kris+kristofferson/not+everyone+knows_20494...,Not everyone knows how sweet your kiss is \nN...,[]
7,Kylie Minogue,Sensitized,/k/kylie+minogue/sensitized_20662603.html,Sensitized by every word that you say \nBoy y...,"[pop, dance, sexy, Kylie Minogue, sensual, aus..."
8,Snoop Dogg,Gangsta Ride,/s/snoop+dogg/gangsta+ride_20127220.html,[Chorus] \nIt's a gangsta gangsta ride \nOn ...,"[Gangsta Rap, West Coast Rap, G-funk]"
9,Alice Cooper,Not That Kind Of Love,/a/alice+cooper/not+that+kind+of+love_20286100...,"Hello, my little pretty \nMy, don't we look Y...","[hard rock, heavy metal, Alice Cooper, Shock R..."


## Saving updated dataframe with tags to a csv. Also removing rows with no tags

In [14]:
df.to_csv('song_tags.csv')

In [15]:
for row_id in df.index:     
    if len(df.loc[row_id,'tags'])==0:
        df = df.drop(row_id)

In [16]:
df

,artist,song,link,text,tags
1,Kiss,Shock Me,/k/kiss/shock+me_20079711.html,Your lightnin's all I need \nMy satisfaction ...,"[hard rock, classic rock, rock, glam rock, 70s..."
2,Ugly Kid Joe,Would You Like To Be There,/u/ugly+kid+joe/would+you+like+to+be+there_101...,"Somewhere the evening sun \nIs falling down, ...","[90s, rock, hard rock, Best song on the album]"
3,Allman Brothers Band,Rockin' Horse,/a/allman+brothers+band/rockin+horse_20765345....,"""My gaurdian angel wears a hard hat,"" \nSaid ...","[Southern Rock, classic rock, blues rock, rock..."
4,Who,The Seeker,/w/who/the+seeker_20146792.html,I've looked under chairs \nI've looked under ...,"[classic rock, rock, 70s, british, hard rock, ..."
7,Kylie Minogue,Sensitized,/k/kylie+minogue/sensitized_20662603.html,Sensitized by every word that you say \nBoy y...,"[pop, dance, sexy, Kylie Minogue, sensual, aus..."
8,Snoop Dogg,Gangsta Ride,/s/snoop+dogg/gangsta+ride_20127220.html,[Chorus] \nIt's a gangsta gangsta ride \nOn ...,"[Gangsta Rap, West Coast Rap, G-funk]"
9,Alice Cooper,Not That Kind Of Love,/a/alice+cooper/not+that+kind+of+love_20286100...,"Hello, my little pretty \nMy, don't we look Y...","[hard rock, heavy metal, Alice Cooper, Shock R..."
10,Yoko Ono,Wake Up,/y/yoko+ono/wake+up_20148771.html,"Mm-mm, mm-mm, mm-mm, \nMm-mm, mm-mm, mm-mm. ...",[General Childrens Music]
11,Gloria Estefan,I See Your Smile,/g/gloria+estefan/i+see+your+smile_20060705.html,I get a little tongue twisted \nEvery time I ...,"[pop, latin, 90s, Love, easy listening, female..."
12,Primus,The Carpenter And The Dainty Bride,/p/primus/the+carpenter+and+the+dainty+bride_2...,"To the carpenter said the dainty bride, \nThe...","[funk metal, alternative, experimental, rock, ..."


In [17]:
df.to_csv('song_tags_updated.csv')

In [15]:
df = pd.read_csv('song_tags_updated copy.csv')

In [16]:
df.head()

,Unnamed: 0,artist,song,link,text,tags
0,1,Kiss,Shock Me,/k/kiss/shock+me_20079711.html,Your lightnin's all I need \nMy satisfaction ...,"['hard rock', 'classic rock', 'rock', 'glam ro..."
1,2,Ugly Kid Joe,Would You Like To Be There,/u/ugly+kid+joe/would+you+like+to+be+there_101...,"Somewhere the evening sun \nIs falling down, ...","['90s', 'rock', 'hard rock', 'Best song on the..."
2,3,Allman Brothers Band,Rockin' Horse,/a/allman+brothers+band/rockin+horse_20765345....,"""My gaurdian angel wears a hard hat,"" \nSaid ...","['Southern Rock', 'classic rock', 'blues rock'..."
3,4,Who,The Seeker,/w/who/the+seeker_20146792.html,I've looked under chairs \nI've looked under ...,"['classic rock', 'rock', '70s', 'british', 'ha..."
4,7,Kylie Minogue,Sensitized,/k/kylie+minogue/sensitized_20662603.html,Sensitized by every word that you say \nBoy y...,"['pop', 'dance', 'sexy', 'Kylie Minogue', 'sen..."


In [59]:
df['tags']

0        ['hard rock', 'classic rock', 'rock', 'glam ro...
1        ['90s', 'rock', 'hard rock', 'Best song on the...
2        ['Southern Rock', 'classic rock', 'blues rock'...
3        ['classic rock', 'rock', '70s', 'british', 'ha...
4        ['pop', 'dance', 'sexy', 'Kylie Minogue', 'sen...
5              ['Gangsta Rap', 'West Coast Rap', 'G-funk']
6        ['hard rock', 'heavy metal', 'Alice Cooper', '...
7                              ['General Childrens Music']
8        ['pop', 'latin', '90s', 'Love', 'easy listenin...
9        ['funk metal', 'alternative', 'experimental', ...
10       ['industrial', 'industrial rock', 'rock', 'NIN...
11       ['country', 'bluegrass', 'favorites', 'america...
12       ['classic rock', 'rock', 'heard on Pandora', '...
13       ['country', 'Kenny Chesney', 'What I Need To D...
14       ['rock', 'alternative rock', 'Ween', 'metal', ...
15       ['halloween', 'Chamo', 'Esta es la que te dije...
16                                          ['Oldies Tag

In [95]:
tags=tags.replace(" ","").split(",")

In [96]:
tags

["['glamrock'",
 "'rock'",
 "'70s'",
 "'classicrock'",
 "'davidbowie'",
 "'glam'",
 "'british'",
 "'hardrock'",
 "'pop'",
 "'sex'",
 "'proto-punk'",
 "'malevocalists'",
 "'bowie'",
 "'harmonica'",
 "'poprock'",
 "'1973'",
 "'hollywood'",
 "'aladdinsane'",
 "'alternative'",
 "'sexy'",
 "'singer-songwriter'",
 "'albumrock'",
 "'party'",
 "'Jam'",
 "'Favourites'",
 "'lovedatfirstlisten'",
 "'tantotempotaste'",
 "'innuendo'",
 "'thesex'",
 "'IconRock'",
 "'jamhot'",
 "'bowieathisbest'",
 "'sleazygoodness'",
 "'oooohsinful'",
 "'80s'",
 "'favoritesongs'",
 "'Progressiverock'",
 "'Love'",
 "'RockRoll'",
 "'blues'",
 "'guitar'",
 "'film'",
 "'artrock'",
 "'fip'",
 "'greatsong'",
 "'Favorite'",
 "'1970s'",
 "'shakespeare'",
 "'asubtleuseofvocalharmony'",
 "'minorkeytonality'",
 "'electricrockinstrumentation'",
 "'Faves'",
 "'musictofuckto'",
 "'Pop/Rock'",
 "'generalrock'",
 "'riff'",
 "'debauchery'",
 "'fuckingcool'",
 "'electricguitarriffs'",
 "'bitchin'",
 '"70\'s"',
 "'fuckingbrilliant'",


## Predefining tags for four moods and adding the same in the dataframe

In [101]:
happyTags = "'cheerful', 'cheer up', 'festive', 'jolly', 'jovial', 'merry', 'cheer', 'cheering',\
'cheery', 'get happy', 'rejoice', 'songs that are cheerful', 'sunny', 'happy', 'happiness', 'happy songs', 'happy music', glad, mood: happy,\
'upbeat', 'gleeful', 'high spirits', 'zest', 'enthusiastic', 'buoyancy', 'elation', 'mood: upbeat','excitement', 'exciting', 'exhilarating', 'thrill',\
'ardor', 'stimulating', 'thrilling', 'titillating', 'christmas', 'christmas song', 'xmas'"
happyTags = happyTags.replace(" ","").split(",")

In [102]:
sadTags = "'sad', 'sadness', 'unhappy', 'melancholic', 'melancholy', 'feeling sad', 'mood: sad - slightly', 'sad song',\
'depressed', 'blue', 'dark', 'depressive', 'dreary', 'gloom', 'darkness', 'depress', 'depression', 'depressing', 'gloomy',\
'grief', 'choleric', 'heartbreak', 'mournful', 'sorrow', 'sorry', 'doleful', 'heartache', 'heartbreaking', 'heartsick', 'lachrymose', 'mourning',\
'plaintive', 'regret', 'sorrowful'"
sadTags = sadTags.replace(" ","").split(",")

In [103]:
loudTags = "'angry', 'anger', 'angry music', 'fury', 'outraged', 'outrage', 'rage', 'death metal', 'thrash metal', 'heavy metal','punk rock', 'hard rock', 'classic rock', 'thrash', 'grunge', 'alternative', 'alternative rock', 'power metal', 'progressive rock','progressive', 'rock', 'rap', 'hiphop', 'metalcore', 'NWOBHM', 'speed metal', 'hell', 'brutal', 'deathcore', 'grindcore', 'black metal','dark', 'industrial', 'metal'"
loudTags = loudTags.replace(" ","").split(",")

In [104]:
calmTags = "'calm', 'relax', 'relaxing', 'slow', 'soft', 'mellow', 'love', 'love song', 'romantic', 'feel good', 'soul', 'duet', 'waltz', 'jazz', 'folk', 'calming', 'quiet', 'sweet', 'melodic', 'easy listening', 'ambient'"
calmTags = calmTags.replace(" ","").split(",")

In [118]:
df['mood']=''

In [119]:
df

,Unnamed: 0,artist,song,link,text,tags,mood
0,1,Kiss,Shock Me,/k/kiss/shock+me_20079711.html,Your lightnin's all I need \nMy satisfaction ...,"['hard rock', 'classic rock', 'rock', 'glam ro...",
1,2,Ugly Kid Joe,Would You Like To Be There,/u/ugly+kid+joe/would+you+like+to+be+there_101...,"Somewhere the evening sun \nIs falling down, ...","['90s', 'rock', 'hard rock', 'Best song on the...",
2,3,Allman Brothers Band,Rockin' Horse,/a/allman+brothers+band/rockin+horse_20765345....,"""My gaurdian angel wears a hard hat,"" \nSaid ...","['Southern Rock', 'classic rock', 'blues rock'...",
3,4,Who,The Seeker,/w/who/the+seeker_20146792.html,I've looked under chairs \nI've looked under ...,"['classic rock', 'rock', '70s', 'british', 'ha...",
4,7,Kylie Minogue,Sensitized,/k/kylie+minogue/sensitized_20662603.html,Sensitized by every word that you say \nBoy y...,"['pop', 'dance', 'sexy', 'Kylie Minogue', 'sen...",
5,8,Snoop Dogg,Gangsta Ride,/s/snoop+dogg/gangsta+ride_20127220.html,[Chorus] \nIt's a gangsta gangsta ride \nOn ...,"['Gangsta Rap', 'West Coast Rap', 'G-funk']",
6,9,Alice Cooper,Not That Kind Of Love,/a/alice+cooper/not+that+kind+of+love_20286100...,"Hello, my little pretty \nMy, don't we look Y...","['hard rock', 'heavy metal', 'Alice Cooper', '...",
7,10,Yoko Ono,Wake Up,/y/yoko+ono/wake+up_20148771.html,"Mm-mm, mm-mm, mm-mm, \nMm-mm, mm-mm, mm-mm. ...",['General Childrens Music'],
8,11,Gloria Estefan,I See Your Smile,/g/gloria+estefan/i+see+your+smile_20060705.html,I get a little tongue twisted \nEvery time I ...,"['pop', 'latin', '90s', 'Love', 'easy listenin...",
9,12,Primus,The Carpenter And The Dainty Bride,/p/primus/the+carpenter+and+the+dainty+bride_2...,"To the carpenter said the dainty bride, \nThe...","['funk metal', 'alternative', 'experimental', ...",


## Assigning 0,1,2,3 and 4 as codes for sad, happy, loud, calm and unknown moods

In [136]:
pbar = pyprind.ProgBar(df.shape[0])
for row_id in df.index:
    tags = df.loc[row_id,'tags']
    tags = tags.replace(" ","").split(",")
    sad_tags = list(set(tags) & set(sadTags))
    happy_tags = list(set(tags) & set(happyTags))
    loud_tags = list(set(tags) & set(loudTags))
    calm_tags = list(set(tags) & set(calmTags))
    if len(sad_tags)>0 or len(happy_tags)>0 or len(loud_tags)>0 or len(calm_tags)>0:# having mood tag
        if len(sad_tags)>len(happy_tags) and len(sad_tags)>len(loud_tags) and len(sad_tags)>len(calm_tags):
            df.loc[row_id,'mood'] = 0
        elif len(happy_tags)>len(sad_tags) and len(happy_tags)>len(loud_tags) and len(happy_tags)>len(calm_tags):
            df.loc[row_id,'mood'] = 1
        elif len(loud_tags)>len(sad_tags) and len(loud_tags)>len(happy_tags) and len(loud_tags)>len(calm_tags):
            df.loc[row_id,'mood'] = 2
        elif len(calm_tags)>len(sad_tags) and len(calm_tags)>len(happy_tags) and len(calm_tags)>len(loud_tags):
            df.loc[row_id,'mood'] = 3
        else:
            df.loc[row_id,'mood'] = 4
    else:
        df = df.drop(row_id)# remove songs that does not have tag
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:07


In [137]:
df

,Unnamed: 0,artist,song,link,text,tags,mood
0,1,Kiss,Shock Me,/k/kiss/shock+me_20079711.html,Your lightnin's all I need \nMy satisfaction ...,"['hard rock', 'classic rock', 'rock', 'glam ro...",2
1,2,Ugly Kid Joe,Would You Like To Be There,/u/ugly+kid+joe/would+you+like+to+be+there_101...,"Somewhere the evening sun \nIs falling down, ...","['90s', 'rock', 'hard rock', 'Best song on the...",2
2,3,Allman Brothers Band,Rockin' Horse,/a/allman+brothers+band/rockin+horse_20765345....,"""My gaurdian angel wears a hard hat,"" \nSaid ...","['Southern Rock', 'classic rock', 'blues rock'...",4
3,4,Who,The Seeker,/w/who/the+seeker_20146792.html,I've looked under chairs \nI've looked under ...,"['classic rock', 'rock', '70s', 'british', 'ha...",2
4,7,Kylie Minogue,Sensitized,/k/kylie+minogue/sensitized_20662603.html,Sensitized by every word that you say \nBoy y...,"['pop', 'dance', 'sexy', 'Kylie Minogue', 'sen...",3
6,9,Alice Cooper,Not That Kind Of Love,/a/alice+cooper/not+that+kind+of+love_20286100...,"Hello, my little pretty \nMy, don't we look Y...","['hard rock', 'heavy metal', 'Alice Cooper', '...",2
8,11,Gloria Estefan,I See Your Smile,/g/gloria+estefan/i+see+your+smile_20060705.html,I get a little tongue twisted \nEvery time I ...,"['pop', 'latin', '90s', 'Love', 'easy listenin...",3
9,12,Primus,The Carpenter And The Dainty Bride,/p/primus/the+carpenter+and+the+dainty+bride_2...,"To the carpenter said the dainty bride, \nThe...","['funk metal', 'alternative', 'experimental', ...",2
10,13,Nine Inch Nails,Throw This Away,/n/nine+inch+nails/throw+this+away_20100948.html,I want you to take me \nI want you to take me...,"['industrial', 'industrial rock', 'rock', 'NIN...",2
11,14,Emmylou Harris,Calling My Children Home,/e/emmylou+harris/calling+my+children+home_200...,Those lives were mine to love and cherish \nT...,"['country', 'bluegrass', 'favorites', 'america...",0


In [138]:
df.to_csv('song_mood.csv')

In [139]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
df.groupby('artist').song.count().plot.bar(ylim=0)
plt.show()


<Figure size 800x600 with 1 Axes>

In [140]:
df

,Unnamed: 0,artist,song,link,text,tags,mood
0,1,Kiss,Shock Me,/k/kiss/shock+me_20079711.html,Your lightnin's all I need \nMy satisfaction ...,"['hard rock', 'classic rock', 'rock', 'glam ro...",2
1,2,Ugly Kid Joe,Would You Like To Be There,/u/ugly+kid+joe/would+you+like+to+be+there_101...,"Somewhere the evening sun \nIs falling down, ...","['90s', 'rock', 'hard rock', 'Best song on the...",2
2,3,Allman Brothers Band,Rockin' Horse,/a/allman+brothers+band/rockin+horse_20765345....,"""My gaurdian angel wears a hard hat,"" \nSaid ...","['Southern Rock', 'classic rock', 'blues rock'...",4
3,4,Who,The Seeker,/w/who/the+seeker_20146792.html,I've looked under chairs \nI've looked under ...,"['classic rock', 'rock', '70s', 'british', 'ha...",2
4,7,Kylie Minogue,Sensitized,/k/kylie+minogue/sensitized_20662603.html,Sensitized by every word that you say \nBoy y...,"['pop', 'dance', 'sexy', 'Kylie Minogue', 'sen...",3
6,9,Alice Cooper,Not That Kind Of Love,/a/alice+cooper/not+that+kind+of+love_20286100...,"Hello, my little pretty \nMy, don't we look Y...","['hard rock', 'heavy metal', 'Alice Cooper', '...",2
8,11,Gloria Estefan,I See Your Smile,/g/gloria+estefan/i+see+your+smile_20060705.html,I get a little tongue twisted \nEvery time I ...,"['pop', 'latin', '90s', 'Love', 'easy listenin...",3
9,12,Primus,The Carpenter And The Dainty Bride,/p/primus/the+carpenter+and+the+dainty+bride_2...,"To the carpenter said the dainty bride, \nThe...","['funk metal', 'alternative', 'experimental', ...",2
10,13,Nine Inch Nails,Throw This Away,/n/nine+inch+nails/throw+this+away_20100948.html,I want you to take me \nI want you to take me...,"['industrial', 'industrial rock', 'rock', 'NIN...",2
11,14,Emmylou Harris,Calling My Children Home,/e/emmylou+harris/calling+my+children+home_200...,Those lives were mine to love and cherish \nT...,"['country', 'bluegrass', 'favorites', 'america...",0


In [3]:
df = pd.read_csv('song_mood.csv')

In [4]:
df

,Unnamed: 0,Unnamed: 0.1,artist,song,link,text,tags,mood
0,0,1,Kiss,Shock Me,/k/kiss/shock+me_20079711.html,Your lightnin's all I need \nMy satisfaction ...,"['hard rock', 'classic rock', 'rock', 'glam ro...",2
1,1,2,Ugly Kid Joe,Would You Like To Be There,/u/ugly+kid+joe/would+you+like+to+be+there_101...,"Somewhere the evening sun \nIs falling down, ...","['90s', 'rock', 'hard rock', 'Best song on the...",2
2,2,3,Allman Brothers Band,Rockin' Horse,/a/allman+brothers+band/rockin+horse_20765345....,"""My gaurdian angel wears a hard hat,"" \nSaid ...","['Southern Rock', 'classic rock', 'blues rock'...",4
3,3,4,Who,The Seeker,/w/who/the+seeker_20146792.html,I've looked under chairs \nI've looked under ...,"['classic rock', 'rock', '70s', 'british', 'ha...",2
4,4,7,Kylie Minogue,Sensitized,/k/kylie+minogue/sensitized_20662603.html,Sensitized by every word that you say \nBoy y...,"['pop', 'dance', 'sexy', 'Kylie Minogue', 'sen...",3
5,6,9,Alice Cooper,Not That Kind Of Love,/a/alice+cooper/not+that+kind+of+love_20286100...,"Hello, my little pretty \nMy, don't we look Y...","['hard rock', 'heavy metal', 'Alice Cooper', '...",2
6,8,11,Gloria Estefan,I See Your Smile,/g/gloria+estefan/i+see+your+smile_20060705.html,I get a little tongue twisted \nEvery time I ...,"['pop', 'latin', '90s', 'Love', 'easy listenin...",3
7,9,12,Primus,The Carpenter And The Dainty Bride,/p/primus/the+carpenter+and+the+dainty+bride_2...,"To the carpenter said the dainty bride, \nThe...","['funk metal', 'alternative', 'experimental', ...",2
8,10,13,Nine Inch Nails,Throw This Away,/n/nine+inch+nails/throw+this+away_20100948.html,I want you to take me \nI want you to take me...,"['industrial', 'industrial rock', 'rock', 'NIN...",2
9,11,14,Emmylou Harris,Calling My Children Home,/e/emmylou+harris/calling+my+children+home_200...,Those lives were mine to love and cherish \nT...,"['country', 'bluegrass', 'favorites', 'america...",0
